In [1]:
using Rx
using BenchmarkTools

In [2]:
function regularArray()
    source = [ i for i in 1:1000 ]
    keep = Array{Int, 1}()
    for value in source
        push!(keep, value)
    end
    return keep
end

regularArray (generic function with 1 method)

In [4]:
struct KeepActor{D} <: NextActor{D}
    values::Array{D, 1}
    
    KeepActor{D}() where D = new(Array{D, 1}())
end

Rx.on_next!(actor::KeepActor{D}, data::D) where D = push!(actor.values, data)

function observableWithKeepActor()
    source = from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(source, keep)
    return keep.values
end

observableWithKeepActor (generic function with 1 method)

In [ ]:
function observableWithLambdaActor()
    source = from([ i for i in 1:1000 ])
    keep = Array{Int, 1}()
    subscribe!(source, LambdaActor{Int}(
        next = (d) -> push!(keep, d)
    ))
    return keep
end

In [5]:
@btime regularArray();
@btime observableWithKeepActor();
@btime observableWithLambdaActor();

  7.373 μs (11 allocations: 24.33 KiB)
  6.736 μs (13 allocations: 24.36 KiB)
  169.458 μs (503 allocations: 32.03 KiB)
